In [1]:
import datetime as dt
import pandas as pd

In [2]:
from fa_tools.PriceDependent import FundamentalPreProc

raw_fundamental = pd.read_csv("./data/cfs_fundamental.csv", index_col=0)

fundamental_preproc = FundamentalPreProc(raw_fundamental)
preproc_fundamental = fundamental_preproc()

In [3]:
from fa_tools.PriceDependent import OhlcvPreProc

raw_ohlcv = pd.read_csv("./data/ohlcv.csv", index_col=0, low_memory=False)
ohlcv_preproc = OhlcvPreProc(raw_ohlcv.rename(columns={"Close": "close"}))
prices = ohlcv_preproc.get_ma_prices(window=32)

In [4]:
from fa_tools.PriceDependent import StockPreProc

raw_stock = pd.read_csv("./data/krx_stocks.csv", index_col=0)
stock_preproc = StockPreProc(raw_stock.rename(columns={"Code": "stock_code", "Stocks": "shares"}))
shares = stock_preproc.get_shares()

In [5]:
from fa_tools.PriceDependent import FaPreProc

fundamental = FaPreProc(preproc_fundamental, prices, shares)()

In [6]:
from fa_tools.PriceDependent import FundamentalProc
fa_proc = FundamentalProc(fundamental)

fa_0 = fa_proc("0")
fa_1 = fa_proc("1")
fa_2 = fa_proc("2")

In [7]:
tpbr_df = pd.concat(
    [
        fa_0["TPBR"].rename("recent_1"),
        fa_1["TPBR"].rename("recent_2"),
        fa_2["TPBR"].rename("recent_3"),
    ],
    axis=1,
)

# TPBR이 감소 한다는 것은 기대치가 낮아지고 있다는 것
_stock_codes_1 = tpbr_df[
    (tpbr_df["recent_1"] < tpbr_df["recent_2"]) & (tpbr_df["recent_2"] < tpbr_df["recent_3"])
].index

In [8]:
net_profit_df = pd.concat(
    [
        fa_0["net_profit"].rename("recent_1"),
        fa_1["net_profit"].rename("recent_2"),
        fa_2["net_profit"].rename("recent_3"),
    ],
    axis=1,
)

# 매년 당기순이익이 증가
_stock_codes_2 = net_profit_df[
    (net_profit_df["recent_3"] < net_profit_df["recent_2"])
    & (net_profit_df["recent_2"] < net_profit_df["recent_1"])
].index

In [9]:
filtered_stocks = set(_stock_codes_1) & set(_stock_codes_2)
len(filtered_stocks)

251

In [10]:
corp_info = pd.read_csv("./data/corp_info_df.csv", index_col=0)
corp_info["stock_code"] = corp_info["stock_code"].apply(lambda x: str(x).zfill(6))

In [11]:
corp_info[corp_info["stock_code"].isin(filtered_stocks)]

,stock_code,corp_name,sector,product
14511,950130,엑세스바이오,의료용품 및 기타 의약 관련제품 제조업,"말라리아 진단키트(RDT), HIV 진단키트(RDT)"
53207,150900,파수,소프트웨어 개발 및 공급업,기업용 문서보안 솔루션
53263,005710,대원산업,자동차 신품 부품 제조업,자동차시트
53508,001820,삼화콘덴서공업,전자부품 제조업,콘덴서(축전기) 제조
53789,023600,삼보판지,"펄프, 종이 및 판지 제조업","골판지,골판지상자"
...,...,...,...,...
103578,049550,잉크테크,기타 화학제품 제조업,"잉크젯프린터용 잉크카트리지, 재충전 잉크"
103602,014440,영보화학,플라스틱제품 제조업,"가교발포폴리에틸렌(아티론,영보드) 제조,판매"
103710,136410,아셈스,플라스틱제품 제조업,핫멜트 접착 필름
103715,348350,위드텍,"측정, 시험, 항해, 제어 및 기타 정밀기기 제조업; 광학기기 제외","AMCs 모니터링 시스템, 공정 프로세스 모니터링 시스템, TMS"
